In [ ]:
#pip install span_marker

In [ ]:
from data_processing import get_subjects_by_Q_json, process_jsonl_and_json
from span_marker import SpanMarkerModel
from tqdm import tqdm
import time
import torch
import json
import numpy as np

spanishfile = '/Users/lolakovalski/Desktop/School/csci375/EAMT/data/spanish_w_labels.json'
italianfile = '/Users/lolakovalski/Desktop/School/csci375/EAMT/data/italian_w_labels.json'

model = SpanMarkerModel.from_pretrained("tomaarsen/span-marker-mbert-base-multinerd", clean_up_tokenization_spaces = True)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [ ]:
with open(spanishfile, 'r') as jf:
    entity_data = [json.loads(line) for line in jf]

eng = entity_data[0]['source']
esp = entity_data[0]['target']
print(esp)

¿Qué actor protagonizó Titanic y nació en Los Ángeles, California?


In [9]:
entities = model.predict(eng)
spanish_entities = model.predict(esp)

In [63]:
print(spanish_entities)
print(entities)

[{'span': 'Titanic', 'label': 'MEDIA', 'score': 0.9936309456825256, 'char_start_index': 23, 'char_end_index': 30}, {'span': 'Los Ángeles', 'label': 'LOC', 'score': 0.9999561309814453, 'char_start_index': 42, 'char_end_index': 53}, {'span': 'California', 'label': 'LOC', 'score': 0.999962329864502, 'char_start_index': 55, 'char_end_index': 65}]
[{'span': 'Titanic', 'label': 'VEHI', 'score': 0.5069388151168823, 'char_start_index': 28, 'char_end_index': 35}, {'span': 'Los Angeles', 'label': 'LOC', 'score': 0.999987006187439, 'char_start_index': 52, 'char_end_index': 63}, {'span': 'California', 'label': 'LOC', 'score': 0.9999725818634033, 'char_start_index': 65, 'char_end_index': 75}]


In [ ]:
line = [entity_data[0]]
lstents = [ki['entities'] for ki in line]

##FROMSEMEVAL
keys = [ent for key in lstents for ent in key]
esents = [key[eskey]['es'] for eskey in keys for key in lstents]
enents = [key[enkey]['en'] for enkey in keys for key in lstents]
print(esents)
print(enents)

##FROM MODEL
print([entity['span'] for entity in spanish_entities])

['Los Ángeles']
['Los Angeles']
['Titanic', 'Los Ángeles', 'California']


In [59]:
test = np.array([entity['span'] for entity in spanish_entities], dtype = object)
print(test)

['Titanic' 'Los Ángeles' 'California']


In [ ]:
true_labels = np.array([
    ["cat", "dog"],         # Observation 1
    ["mouse"],              # Observation 2
    ["elephant", "rabbit"]  # Observation 3
], dtype=object)

predicted_labels = np.array([
    ["dog", "cat"],         # Prediction for Observation 1
    ["mouse", "cat"],       # Prediction for Observation 2
    ["rabbit"]              # Prediction for Observation 3
], dtype=object)

# Initialize counters
TP, FP, FN = 0, 0, 0

# Loop through each observation
for true, pred in zip(true_labels, predicted_labels):
    true_set = set(true)
    pred_set = set(pred)
    
    TP += len(true_set & pred_set)  # Intersection: correctly predicted
    FP += len(pred_set - true_set)  # Predicted but not true
    FN += len(true_set - pred_set)  # True but not predicted

# Calculate metrics
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 0.80
Recall: 0.80
F1 Score: 0.80


In [12]:
import os
import json
with open(os.path.abspath('data/spanish_test.json'), 'r', encoding='utf-8') as jf:

    inputs = json.load(jf)
    #inputs = [json.loads(line) for line in jf]
print(inputs[0])

{'id': '25ed319e', 'source_locale': 'en', 'target_locale': 'es', 'source': 'What is the largest park in the capital city of the UK?', 'target': '¿Cuál es el parque más grande en la capital del Reino Unido?', 'entities': {'Q145': {'es': 'Reino Unido', 'en': 'United Kingdom'}}, 'from': 'mintaka'}


In [ ]:
# Import necessary functions
import os
import json
import random
from entity_masking import load_entities_mapping, mask_entities_with_spanmarker, ensure_pad_token
from mt_translation import translate_with_placeholders_m2m
from entity_reintegration import reintegrate_entities
from ner_processing import sanity_check, identify_named_entities
    
es_train = os.path.abspath('data/spanish_test.json')
it_train = os.path.abspath('data/italian_test.json')


print("Loading data... \n")

nerRes = {"es": None,"it": None}

# Step 2: Perform Named Entity Recognition (NER)
for language in ['es', 'it']:
    print("Identifying named entities {}... \n".format(language))
    file = es_train if language == 'es' else it_train
    with open(file, 'r', encoding='utf-8') as jf:
        inputs = json.load(jf)

    nerRes[language] = identify_named_entities(inputs, language, "test")
    input = {"entities_labeled": nerRes['target_entities_labeled'], "entities_predicted": nerRes['target_entities_predicted']}

    precision, recall, F1 = sanity_check(input)
    print("Accuracy of SpanMarker for Multilingual Named Entity Recognition {}\n".format(language))
    print("Precision: {}".format(precision))
    print("Recall: {}".format(recall))
    print("F1: {}".format(F1))


Loading data... 

Identifying named entities es... 



The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
Processing source sentences:  44%|████▍     | 133/299 [25:12<31:02, 11.22s/it] 